# SQL. Подзапросы. Табличные выражения. Оконные функции.


Одним из часто используемых и важных инструментов при написании SQL-запросов являются подзапросы.

**Подзапрос** — это запрос, использующийся в другом SQL запросе. Подзапрос всегда заключён в круглые скобки и обычно выполняется перед основным запросом.

Подзапрос может возвращать:

* одна строка и один столбец;

* нескольких строк с одним столбцом;

* нескольких строк с несколькими столбцами.

В зависимости от типа результирующего набора подзапроса определяются операторы, которые могут использоваться в основном запросе.

Подзапрос используется для получения данных, которые будут использованы основным запросом в качестве условия для фильтрации возвращаемых записей.

Подзапросы могут использоваться в инструкциях SELECT, INSERT, UPDATE и DELETE, а также с операторами =, <, >, >=, <=, IN, BETWEEN и т.д.



In [ ]:
SELECT name, age FROM Users
WHERE age = (SELECT max(age) FROM Users)

SELECT name, age FROM Users
WHERE age in (SELECT age FROM Users WHERE name LIKE 'a%')

## Табличные выражения

**Обобщённое табличное выражение** или CTE (Common Table Expressions) - это временный результирующий набор данных, к которому можно обращаться в последующих запросах. Для написания обобщённого табличного выражения используется оператор WITH.

<code>
-- Пример использования конструкции WITH
WITH Aeroflot_trips AS
    (SELECT TRIP.* FROM Company
        INNER JOIN Trip ON Trip.company = Company.id WHERE name = "Aeroflot")

SELECT plane, COUNT(plane) AS amount FROM Aeroflot_trips GROUP BY plane;
</code>

Выражение с **WITH** считается «временным», потому что результат не сохраняется где-либо на постоянной основе в схеме базы данных, а действует как временное представление, которое существует только на время выполнения запроса, то есть оно доступно только во время выполнения операторов SELECT, INSERT, UPDATE, DELETE или MERGE. Оно действительно только в том запросе, которому он принадлежит, что позволяет улучшить структуру запроса, не загрязняя глобальное пространство имён.


## Оконные функции

Рассмотрим оконные функции на примере данных о конверсии по разным каналам привлечения клиентов.

Для справки:

* cpc  - контекстная реклама

* organic – бесплатная поисковая выдача

* direct - прямые переходы на сайт (пользователь в браузере вбил адрес сайта)

* cpa - модель оплаты рекламы, при которой учитывается стоимость за выполнение определенных действий на сайте.

In [1]:
import pandas as pd
import numpy as np

import sqlite3

In [2]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [3]:
engine = create_connection('SQL_14_07.db')

Connection to SQLite DB successful


In [4]:
d = {'Date': ['18.03.2024', '18.03.2024','18.03.2024','19.03.2024','19.03.2024','19.03.2024','19.03.2024','19.03.2024','20.03.2024','20.03.2024','21.03.2024'], 'Medium': ['cpa', 'cpc', 'organic', 'cpa', 'cpc', 'organic', 'direct', 'cpc', 'organic','direct', 'cpc'], 'Conversions': [1, 2, 4, 5, 1, 3, 2, 4, 2, 5, 3]}

df1 = pd.DataFrame(d)
df1

,Date,Medium,Conversions
0,18.03.2024,cpa,1
1,18.03.2024,cpc,2
2,18.03.2024,organic,4
3,19.03.2024,cpa,5
4,19.03.2024,cpc,1
5,19.03.2024,organic,3
6,19.03.2024,direct,2
7,19.03.2024,cpc,4
8,20.03.2024,organic,2
9,20.03.2024,direct,5


In [5]:
df1.to_sql('conversion', engine, if_exists='replace', index=False)

11

In [6]:
df1

,Date,Medium,Conversions
0,18.03.2024,cpa,1
1,18.03.2024,cpc,2
2,18.03.2024,organic,4
3,19.03.2024,cpa,5
4,19.03.2024,cpc,1
5,19.03.2024,organic,3
6,19.03.2024,direct,2
7,19.03.2024,cpc,4
8,20.03.2024,organic,2
9,20.03.2024,direct,5


In [10]:
query = '''SELECT Medium FROM conversion GROUP BY Medium HAVING AVG(Conversions) > 3
        '''
db = pd.read_sql_query(sql = query, con = engine)
db

,Medium
0,direct


In [11]:
query = '''SELECT DISTINCT Medium, AVG(Conversions) as conv_mean
    FROM conversion
    WHERE Medium IN (SELECT Medium FROM conversion GROUP BY Medium HAVING AVG(Conversions) > 2)
        '''
db = pd.read_sql_query(sql = query, con = engine)
db

,Medium,conv_mean
0,cpa,2.909091


Откроем окно при помощи OVER() и просуммируем столбец c конверсиями:

In [12]:
query = '''SELECT
      Date,
      Medium,
      Conversions,
      SUM(Conversions) OVER() AS 'Sum'
    FROM conversion
        '''
db = pd.read_sql_query(sql = query, con = engine)
db

,Date,Medium,Conversions,Sum
0,18.03.2024,cpa,1,32
1,18.03.2024,cpc,2,32
2,18.03.2024,organic,4,32
3,19.03.2024,cpa,5,32
4,19.03.2024,cpc,1,32
5,19.03.2024,organic,3,32
6,19.03.2024,direct,2,32
7,19.03.2024,cpc,4,32
8,20.03.2024,organic,2,32
9,20.03.2024,direct,5,32


In [14]:
query = '''SELECT
      Date,
      Medium,
      Conversions,
      SUM(Conversions) OVER() AS 'Sum',
      AVG(Conversions) OVER() AS conv_mean
    FROM conversion
        '''
db = pd.read_sql_query(sql = query, con = engine)
db

,Date,Medium,Conversions,Sum,conv_mean
0,18.03.2024,cpa,1,32,2.909091
1,18.03.2024,cpc,2,32,2.909091
2,18.03.2024,organic,4,32,2.909091
3,19.03.2024,cpa,5,32,2.909091
4,19.03.2024,cpc,1,32,2.909091
5,19.03.2024,organic,3,32,2.909091
6,19.03.2024,direct,2,32,2.909091
7,19.03.2024,cpc,4,32,2.909091
8,20.03.2024,organic,2,32,2.909091
9,20.03.2024,direct,5,32,2.909091


In [16]:
query = '''SELECT Date,
      Medium,
      Conversions,
      conv_mean,
      sum_conv,
      Conversions - conv_mean as diff_conv
      FROM (SELECT
      Date,
      Medium,
      Conversions,
      SUM(Conversions) OVER() AS sum_conv,
      AVG(Conversions) OVER() AS conv_mean
    FROM conversion)
        '''
db = pd.read_sql_query(sql = query, con = engine)
db

,Date,Medium,Conversions,conv_mean,sum_conv,diff_conv
0,18.03.2024,cpa,1,2.909091,32,-1.909091
1,18.03.2024,cpc,2,2.909091,32,-0.909091
2,18.03.2024,organic,4,2.909091,32,1.090909
3,19.03.2024,cpa,5,2.909091,32,2.090909
4,19.03.2024,cpc,1,2.909091,32,-1.909091
5,19.03.2024,organic,3,2.909091,32,0.090909
6,19.03.2024,direct,2,2.909091,32,-0.909091
7,19.03.2024,cpc,4,2.909091,32,1.090909
8,20.03.2024,organic,2,2.909091,32,-0.909091
9,20.03.2024,direct,5,2.909091,32,2.090909


In [18]:
query = '''WITH agg_conv as
        (SELECT
          Date,
          Medium,
          Conversions,
          SUM(Conversions) OVER() AS sum_conv,
          AVG(Conversions) OVER() AS conv_mean
        FROM conversion)
        SELECT Date,
          Medium,
          Conversions,
          conv_mean,
          sum_conv,
          Conversions - conv_mean as diff_conv
          FROM agg_conv
        '''
db = pd.read_sql_query(sql = query, con = engine)
db

,Date,Medium,Conversions,conv_mean,sum_conv,diff_conv
0,18.03.2024,cpa,1,2.909091,32,-1.909091
1,18.03.2024,cpc,2,2.909091,32,-0.909091
2,18.03.2024,organic,4,2.909091,32,1.090909
3,19.03.2024,cpa,5,2.909091,32,2.090909
4,19.03.2024,cpc,1,2.909091,32,-1.909091
5,19.03.2024,organic,3,2.909091,32,0.090909
6,19.03.2024,direct,2,2.909091,32,-0.909091
7,19.03.2024,cpc,4,2.909091,32,1.090909
8,20.03.2024,organic,2,2.909091,32,-0.909091
9,20.03.2024,direct,5,2.909091,32,2.090909


Инструкция **OVER()** использована без предложений. В таком варианте окном будет весь набор данных и никакая сортировка не применяется. В итоге получаем в новом столбце сумму по всем строкам по полю Conversions.

Теперь применим инструкцию **PARTITION BY**, которая определяет столбец, по которому будет производиться группировка и является ключевой в разделении набора строк на окна:

In [19]:
query = '''SELECT
        Date,
        Medium,
        Conversions,
        SUM(Conversions) OVER(PARTITION BY Date) AS sum_conv
        FROM conversion
        '''
db = pd.read_sql_query(sql = query, con = engine)
db

,Date,Medium,Conversions,sum_conv
0,18.03.2024,cpa,1,7
1,18.03.2024,cpc,2,7
2,18.03.2024,organic,4,7
3,19.03.2024,cpa,5,15
4,19.03.2024,cpc,1,15
5,19.03.2024,organic,3,15
6,19.03.2024,direct,2,15
7,19.03.2024,cpc,4,15
8,20.03.2024,organic,2,7
9,20.03.2024,direct,5,7


Инструкция PARTITION BY сгруппировала строки по полю «Date». Теперь для каждой группы рассчитывается своя сумма значений столбца «Conversions».

Отсортируем значения внутри окна при помощи **ORDER BY**:

In [20]:
query = '''SELECT
        Date,
        Medium,
        Conversions,
        SUM(Conversions) OVER(PARTITION BY Date ORDER BY Medium) AS sum_conv
        FROM conversion
        '''
db = pd.read_sql_query(sql = query, con = engine)
db

,Date,Medium,Conversions,sum_conv
0,18.03.2024,cpa,1,1
1,18.03.2024,cpc,2,3
2,18.03.2024,organic,4,7
3,19.03.2024,cpa,5,5
4,19.03.2024,cpc,1,10
5,19.03.2024,cpc,4,10
6,19.03.2024,direct,2,12
7,19.03.2024,organic,3,15
8,20.03.2024,direct,5,5
9,20.03.2024,organic,2,7


К предложению PARTITION BY добавилось ORDER BY по полю «Medium». Таким образом мы указали, что хотим видеть сумму не всех значений в окне, а для каждого значения «Conversions» сумму со всеми предыдущими. То есть мы посчитали нарастающий итог.

### ROWS и RANGE

Инструкция **ROWS** позволяет ограничить строки в окне, указывая фиксированное количество строк, предшествующих или следующих за текущей.

Инструкция **RANGE**, в отличие от **ROWS**, работает не со строками, а с диапазоном строк в инструкции ORDER BY. То есть под одной строкой для RANGE могут пониматься несколько физических строк одинаковых по рангу.

Обе инструкции **ROWS** и **RANGE** всегда используются вместе с **ORDER BY**.

В выражении для ограничения строк ROWS или RANGE также можно использовать следующие ключевые слова:

* UNBOUNDED PRECEDING — указывает, что окно начинается с первой строки группы

* UNBOUNDED FOLLOWING – с помощью данной инструкции можно указать, что окно заканчивается на последней строке группы

* CURRENT ROW – инструкция указывает, что окно начинается или заканчивается на текущей строке

* BETWEEN «граница окна» AND «граница окна» — указывает нижнюю и верхнюю границу окна

* «Значение» PRECEDING – определяет число строк перед текущей строкой (не допускается в предложении RANGE)

* «Значение» FOLLOWING — определяет число строк после текущей строки (не допускается в предложении RANGE)

In [22]:
query = '''SELECT
        Date,
        Medium,
        Conversions,
        SUM(Conversions) OVER(PARTITION BY Date ORDER BY Conversions ROWS BETWEEN CURRENT ROW AND 2 FOLLOWING) AS 'Sum'
        FROM conversion
        '''
db = pd.read_sql_query(sql = query, con = engine)
db

,Date,Medium,Conversions,Sum
0,18.03.2024,cpa,1,7
1,18.03.2024,cpc,2,6
2,18.03.2024,organic,4,4
3,19.03.2024,cpc,1,6
4,19.03.2024,direct,2,9
5,19.03.2024,organic,3,12
6,19.03.2024,cpc,4,9
7,19.03.2024,cpa,5,5
8,20.03.2024,organic,2,7
9,20.03.2024,direct,5,5


В данном случае сумма рассчитывается по текущей и следующей ячейке в окне. А последняя строка в окне имеет то же значение, что и столбец «Conversions», потому что больше не с чем складывать.

### Виды оконных функций

Выделяют следующие типы оконных функций:

* Агрегатные функции

* Ранжирующие функции

* Функции смещения

* Аналитические функции

### Агрегатные функции

Агрегатные функции – это функции, которые выполняют на наборе данных арифметические вычисления и возвращают итоговое значение.

* SUM – возвращает сумму значений в столбце;

* COUNT — вычисляет количество значений в столбце (значения NULL не учитываются);

* AVG — определяет среднее значение в столбце;

* MAX — определяет максимальное значение в столбце;

* MIN — определяет минимальное значение в столбце.

In [23]:
query = '''SELECT
        Date,
        Medium,
        Conversions,
        SUM(Conversions) OVER(PARTITION BY Date) AS sum_conv,
        COUNT(Conversions) OVER(PARTITION BY Date) AS count_conv,
        AVG(Conversions) OVER(PARTITION BY Date) AS mean_conv,
        MAX(Conversions) OVER(PARTITION BY Date) AS max_conv,
        MIN(Conversions) OVER(PARTITION BY Date) AS min_conv
        FROM conversion
        '''
db = pd.read_sql_query(sql = query, con = engine)
db

,Date,Medium,Conversions,sum_conv,count_conv,mean_conv,max_conv,min_conv
0,18.03.2024,cpa,1,7,3,2.333333,4,1
1,18.03.2024,cpc,2,7,3,2.333333,4,1
2,18.03.2024,organic,4,7,3,2.333333,4,1
3,19.03.2024,cpa,5,15,5,3.000000,5,1
4,19.03.2024,cpc,1,15,5,3.000000,5,1
5,19.03.2024,organic,3,15,5,3.000000,5,1
6,19.03.2024,direct,2,15,5,3.000000,5,1
7,19.03.2024,cpc,4,15,5,3.000000,5,1
8,20.03.2024,organic,2,7,2,3.500000,5,2
9,20.03.2024,direct,5,7,2,3.500000,5,2


### Ранжирующие функции

Ранжирующие функции – это функции, которые ранжируют значение для каждой строки в окне. Например, их можно использовать для того, чтобы присвоить порядковый номер строке или составить рейтинг.

* **ROW_NUMBER** – функция возвращает номер строки и используется для нумерации;

* **RANK** — функция возвращает ранг каждой строки. В данном случае значения уже анализируются и, в случае нахождения одинаковых, возвращает одинаковый ранг с пропуском следующего значения;

* **DENSE_RANK** — функция возвращает ранг каждой строки. Но в отличие от функции RANK, она для одинаковых значений возвращает ранг, не пропуская следующий;

* **NTILE** – это функция, которая позволяет определить к какой группе относится текущая строка. Количество групп задается в скобках.

In [24]:
query = '''SELECT
        Date,
        Medium,
        Conversions,
        ROW_NUMBER() OVER(PARTITION BY Date ORDER BY Conversions) AS Row_number_conv,
        RANK() OVER(PARTITION BY Date ORDER BY Conversions) AS Rank_conv,
        DENSE_RANK() OVER(PARTITION BY Date ORDER BY Conversions) AS Dense_Rank_conv,
        NTILE(3) OVER(PARTITION BY Date ORDER BY Conversions) AS Ntile_conv
        FROM conversion
        '''
db = pd.read_sql_query(sql = query, con = engine)
db

,Date,Medium,Conversions,Row_number_conv,Rank_conv,Dense_Rank_conv,Ntile_conv
0,18.03.2024,cpa,1,1,1,1,1
1,18.03.2024,cpc,2,2,2,2,2
2,18.03.2024,organic,4,3,3,3,3
3,19.03.2024,cpc,1,1,1,1,1
4,19.03.2024,direct,2,2,2,2,1
5,19.03.2024,organic,3,3,3,3,2
6,19.03.2024,cpc,4,4,4,4,2
7,19.03.2024,cpa,5,5,5,5,3
8,20.03.2024,organic,2,1,1,1,1
9,20.03.2024,direct,5,2,2,2,2


### Функции смещения

Функции смещения – это функции, которые позволяют перемещаться и обращаться к разным строкам в окне, относительно текущей строки, а также обращаться к значениям в начале или в конце окна.

* **LAG**/**LEAD** – функция LAG обращается к данным из предыдущей строки окна, а LEAD к данным из следующей строки. Функцию можно использовать для того, чтобы сравнивать текущее значение строки с предыдущим или следующим. Имеет три параметра: столбец, значение которого необходимо вернуть, количество строк для смещения (по умолчанию 1), значение, которое необходимо вернуть если после смещения возвращается значение NULL;

* **FIRST_VALUE**/**LAST_VALUE** — с помощью функции можно получить первое/последнее значение в окне. В качестве параметра принимает столбец, значение которого необходимо вернуть.

In [25]:
query = '''SELECT
        Date,
        Medium,
        Conversions,
        LAG(Conversions) OVER(PARTITION BY Date ORDER BY Date) AS Lag_conv,
        LEAD(Conversions) OVER(PARTITION BY Date ORDER BY Date) AS Lead_conv,
        FIRST_VALUE(Conversions) OVER(PARTITION BY Date ORDER BY Date) AS First_Value_conv,
        LAST_VALUE(Conversions) OVER(PARTITION BY Date ORDER BY Date) AS Last_Value_conv
        FROM conversion
        '''
db = pd.read_sql_query(sql = query, con = engine)
db

,Date,Medium,Conversions,Lag_conv,Lead_conv,First_Value_conv,Last_Value_conv
0,18.03.2024,cpa,1,NaN,2.0,1,4
1,18.03.2024,cpc,2,1.0,4.0,1,4
2,18.03.2024,organic,4,2.0,NaN,1,4
3,19.03.2024,cpa,5,NaN,1.0,5,4
4,19.03.2024,cpc,1,5.0,3.0,5,4
5,19.03.2024,organic,3,1.0,2.0,5,4
6,19.03.2024,direct,2,3.0,4.0,5,4
7,19.03.2024,cpc,4,2.0,NaN,5,4
8,20.03.2024,organic,2,NaN,5.0,2,5
9,20.03.2024,direct,5,2.0,NaN,2,5


In [26]:
query = '''SELECT
        Date,
        Medium,
        Conversions,
        LAG(Conversions, 1, 10) OVER(PARTITION BY Date ORDER BY Date) AS Lag_conv,
        LEAD(Conversions) OVER(PARTITION BY Date ORDER BY Date) AS Lead_conv,
        FIRST_VALUE(Conversions) OVER(PARTITION BY Date ORDER BY Date) AS First_Value_conv,
        LAST_VALUE(Conversions) OVER(PARTITION BY Date ORDER BY Date) AS Last_Value_conv
        FROM conversion
        '''
db = pd.read_sql_query(sql = query, con = engine)
db

,Date,Medium,Conversions,Lag_conv,Lead_conv,First_Value_conv,Last_Value_conv
0,18.03.2024,cpa,1,10,2.0,1,4
1,18.03.2024,cpc,2,1,4.0,1,4
2,18.03.2024,organic,4,2,NaN,1,4
3,19.03.2024,cpa,5,10,1.0,5,4
4,19.03.2024,cpc,1,5,3.0,5,4
5,19.03.2024,organic,3,1,2.0,5,4
6,19.03.2024,direct,2,3,4.0,5,4
7,19.03.2024,cpc,4,2,NaN,5,4
8,20.03.2024,organic,2,10,5.0,2,5
9,20.03.2024,direct,5,2,NaN,2,5


### Аналитические функции

Аналитические функции — это функции которые возвращают информацию о распределении данных и используются для статистического анализа.

* **CUME_DIST** — вычисляет интегральное распределение (относительное положение) значений в окне;

* **PERCENT_RANK** — вычисляет относительный ранг строки в окне;

* **PERCENTILE_CONT** — вычисляет процентиль на основе постоянного распределения значения столбца. В качестве параметра принимает процентиль, который необходимо вычислить;

* **PERCENTILE_DISC** — вычисляет определенный процентиль для отсортированных значений в наборе данных. В качестве параметра принимает процентиль, который необходимо вычислить.

In [27]:
query = '''SELECT
        Date,
        Medium,
        Conversions,
        CUME_DIST() OVER(PARTITION BY Date ORDER BY Conversions) AS Cume_Dist_conv,
        PERCENT_RANK() OVER(PARTITION BY Date ORDER BY Conversions) AS Percent_Rank_conv
        FROM conversion
        '''
db = pd.read_sql_query(sql = query, con = engine)
db

,Date,Medium,Conversions,Cume_Dist_conv,Percent_Rank_conv
0,18.03.2024,cpa,1,0.333333,0.00
1,18.03.2024,cpc,2,0.666667,0.50
2,18.03.2024,organic,4,1.000000,1.00
3,19.03.2024,cpc,1,0.200000,0.00
4,19.03.2024,direct,2,0.400000,0.25
5,19.03.2024,organic,3,0.600000,0.50
6,19.03.2024,cpc,4,0.800000,0.75
7,19.03.2024,cpa,5,1.000000,1.00
8,20.03.2024,organic,2,0.500000,0.00
9,20.03.2024,direct,5,1.000000,1.00


Источник: https://thisisdata.ru/blog/uchimsya-primenyat-okonnyye-funktsii/

Дополнительно попрактиковаться с запросами можно в тренажёре: https://www.w3schools.com/sql/

## Практические задания

1. Загрузите данные из датасета: https://www.kaggle.com/datasets/nelgiriyewithana/global-weather-repository

Найти максимальное значение среди средних температур по странам, посчитанных для каждой страны отдельно (т. е. группировка при расчёте средней температуры по странам).

2. Напишите SQL-запрос, который выбирает страны города которых не попадают в топ-10 по наиболее высокой температуре.

3. Для каждой из стран вычислите разницу средней температуры по стране и средней температуры по всем странам. Помимо этого, вычислите на сколько процентов средняя температура по стране отличается от средней температуры по всем странам.

4. Выберите записи из таблицы Weather, которые были обновлены за последнюю дата (поле last_updated) присутствующую в данных. Из этих записей создайте таблицу в которой будет страны, города, температура и средняя температура рассчитанная по всем данных и по данным за последнюю дату.
